In [1]:
!pip install netron

import torch
import torch.nn as nn
from edgeai_torchmodelopt import xmodelopt
import copy
import netron
import torchvision
from tqdm import tqdm

/home/a0491009/.pyenv/versions/3.10.13/envs/quant/lib/python3.10/site-packages/tqdm-4.66.2-py3.10.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = torchvision.models.resnet50(weights='DEFAULT')
example_input = torch.rand((1, 3, 224, 224))

y = model(example_input)
print("Output Shape is : {}".format(y.shape))

Output Shape is : torch.Size([1, 1000])


In [3]:
model_export_name = "./orig_simple_network_ptc.onnx"
torch.onnx.export(model, example_input, model_export_name)
netron.start(model_export_name, 8080)

Serving './orig_simple_network_ptc.onnx' at http://localhost:8080


('localhost', 8080)

In [4]:
from edgeai_torchmodelopt.xmodelopt.surgery.v3 import default_replacement_flag_dict
print("The surgery provides some default replacements which can be used as is. Check the \n \
      mapping in 'flag_to_dict_entries' in 'surgery.v3.__init__'. ")
default_replacement_flag_dict

The surgery provides some default replacements which can be used as is. Check the 
       mapping in 'flag_to_dict_entries' in 'surgery.v3.__init__'. 


{'squeeze_and_excite_to_identity': True,
 'all_activation_to_relu': False,
 'relu_inplace_to_relu': True,
 'gelu_to_relu': True,
 'relu6_to_relu': True,
 'silu_to_relu': True,
 'hardswish_to_relu': True,
 'hardsigmoid_to_relu': True,
 'leakyrelu_to_relu': True,
 'dropout_inplace_to_dropout': True,
 'break_maxpool2d_with_kernel_size_greater_than_equalto_5': True,
 'break_avgpool2d_with_kernel_size_greater_than_equalto_5': True,
 'convert_resize_params_size_to_scale': True,
 'promote_conv2d_with_even_kernel_to_larger_odd_kernel': False,
 'break_conv2d_with_kernel_size_greater_than_7': False}

In [5]:
transformation_dict = None # provides the functionality to use the toolkit separately for each module of the model. 
                           # the transformation dict can be specified as dict(backbone=None, neck=None, bbox_head=None) 
                           # in mmdet or similar use cases
qconfig_type = "DEFAULT" #MSA_WC8_AT8, WC8_AT8, WC8SYMP2_AT8SYMP2 # can be obtained from 'quantization.v3.qconfig_types'.
copy_attrs = []
example_inputs = [example_input]
example_kwargs = {}
# Both are supported formats for surgery_kwargs, can provide bool for the already defined mappings, else, 
# the mapping can be genarated and provided. The example for custom surgery function can be obtained from 
# 'gen_func_for_conv2d_even_kernel_to_odd' function from 'surgery.v3.custom_surgery_functions'.
surgery_kwargs = dict(replacement_dict = 
                      {'relu_to_gelu' : {nn.ReLU: nn.GELU},
                       'hardswish_to_relu' : True}
                      )
quantization_kwargs = dict(quantization_method='PTC', total_epochs=3, qconfig_type=qconfig_type, bias_calibration_factor=0.01, 
                           num_observer_update_epochs = 2, num_batch_norm_update_epochs = 1)
            
model = xmodelopt.apply_model_optimization(model, example_inputs, example_kwargs, model_surgery_version=3, \
                quantization_version=3, model_surgery_kwargs=surgery_kwargs, quantization_kwargs=quantization_kwargs, 
                transformation_dict=transformation_dict, copy_attrs=copy_attrs)
            

/home/a0491009/quantization/edgeai-modeloptimization/torchmodelopt/edgeai_torchmodelopt/xmodelopt/utils/optimization_base.py:10: UserWarning: OptimizationBaseModule is a class. Some of the attributes (non-methods) in 'attrs' may not be of a class. They may be attributes of an object. If they are passed, they will be discarded.
  warnings.warn(f"{src.__name__} is a class. Some of the attributes (non-methods) in 'attrs' may not be of a class. They may be attributes of an object. If they are passed, they will be discarded.")


Dynamo Export Completed ! 


Model Preparation is now complete! 


/home/a0491009/quantization/edgeai-modeloptimization/torchmodelopt/edgeai_torchmodelopt/xmodelopt/utils/optimization_base.py:10: UserWarning: OptimizationBaseModule is a class. Some of the attributes (non-methods) in 'attrs' may not be of a class. They may be attributes of an object. If they are passed, they will be discarded.
  warnings.warn(f"{src.__name__} is a class. Some of the attributes (non-methods) in 'attrs' may not be of a class. They may be attributes of an object. If they are passed, they will be discarded.")


In [6]:
num_calib_images = 10
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for i in tqdm(range(num_calib_images)):
        output = model(torch.rand(1,3,224,224))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:05<00:00,  1.76it/s]


Freezing BN for subsequent epochs


100%|██████████| 10/10 [00:05<00:00,  1.67it/s]


Freezing ranges for subsequent epochs


100%|██████████| 10/10 [00:01<00:00,  6.22it/s]


In [7]:
model_export_name = "./converted_simple_network_ptc.onnx"
model.export(example_input, model_export_name)
netron.start(model_export_name, 8080)


W1023 16:32:09.577000 138670485233152 torch/_export/__init__.py:95] +============================+
W1023 16:32:09.579000 138670485233152 torch/_export/__init__.py:96] |     !!!   WARNING   !!!    |
W1023 16:32:09.579000 138670485233152 torch/_export/__init__.py:97] +============================+
W1023 16:32:09.580000 138670485233152 torch/_export/__init__.py:98] capture_pre_autograd_graph() is deprecated and doesn't provide any function guarantee moving forward.
W1023 16:32:09.581000 138670485233152 torch/_export/__init__.py:99] Please switch to use torch.export instead.


The loss branch is not getting removed in the model, exporting normally.


/home/a0491009/quantization/edgeai-modeloptimization/torchmodelopt/edgeai_torchmodelopt/xmodelopt/quantization/v3/quant_func.py:302: UserWarning: model has already been converted before calling export. make sure it is done correctly.
  warnings.warn("model has already been converted before calling export. make sure it is done correctly.")
/home/a0491009/.pyenv/versions/3.10.13/envs/quant/lib/python3.10/site-packages/torch/onnx/utils.py:860: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn(
[W1023 16:32:13.590851751 shape_type_infer

Stopping http://localhost:8080
Serving './converted_simple_network_ptc.onnx' at http://localhost:8080


('localhost', 8080)